In [4]:
import cadquery as cq
from jupyter_cadquery import (
    show, close_viewer, close_viewers, get_viewer, open_viewer, set_defaults, get_defaults
)

from jupyter_cadquery.replay import replay, enable_replay, disable_replay, reset_replay

set_defaults(axes=False, timeit=False)
enable_replay(show_bbox=True, warning=False)
show_object = replay


Enabling jupyter_cadquery replay


In [7]:
import cadquery as cq
import math

height = 16
hyp = 180
leg = hyp / math.sqrt(2)
corner_fillet = leg / 20
edge_fillet = height/3

print(f"hyp={hyp} leg={leg} height={height}")

s = (
    cq.Sketch()
    .segment((0.0, 0), (0.0, leg))
    .segment((leg, 0))
    .close()
    .assemble(tag="face")
    .edges("%LINE", tag="face")
    .vertices()
    .fillet(corner_fillet)
)

box = cq.Workplane("XY").placeSketch(s).extrude(height).fillet(0.9999*edge_fillet)
# fontsize = (height - 2*edge_fillet )/2
# sig = (
#     box.faces(">(1, 1, 0)" ).workplane()
#     .center(0,height/2)
#     .text("hello world", fontsize=fontsize, distance=-1, fontPath="Arial.ttf",
#          halign="center", valign="center") # Specify font path
# )
# show_object(sig, show_result=True)
# box = sig


hyp=180 leg=127.27922061357854 height=16


In [8]:
hole_depth = height/2
hole_sink = hole_depth / 4
hole_width = height / 4

h_sep = height*0.75
mesa = h_sep - 2*hole_width
if mesa < edge_fillet:
    h_sep = edge_fillet + 2*hole_width
    mesa  = edge_fillet

num_holes = 1+2*(math.floor((leg-2*edge_fillet) / h_sep) - 1)
# num_holes=3



print(f"Using hsep={h_sep}, mesa={mesa}, hole_sink={hole_sink}, edge_fillet={edge_fillet}")
result = (
    box.faces("<Y")    
    .workplane()
    .center(hole_width + edge_fillet, hole_sink) # orig
    # .center(hole_width + edge_fillet, height-hole_sink) # new
    .rarray(h_sep, h_sep, num_holes, 1, True)
    .ellipse(hole_width, hole_depth)
    .cutThruAll()
        
    .faces("<X")
    .workplane()
    .center(-1*(hole_width+edge_fillet), height-2*hole_sink) #orig
    # .center(-hole_width- edge_fillet, hole_sink) #new
    .rarray(h_sep, h_sep, num_holes, 1, True)
    .ellipse(hole_width, hole_depth)
    .cutThruAll()
)

# resuls = result+sig

show_object(result, show_result=True)
# show(result, viewer="SideCar", anchor="right")


Using hsep=13.333333333333332, mesa=5.333333333333333, hole_sink=2.0, edge_fillet=5.333333333333333
Use the multi select box below to select one or more steps you want to examine
+

In [9]:
from cadquery import exporters

rresult = result.mirror(mirrorPlane="ZY", basePointVector=(-height, 0, 0))
show_object(rresult, show_result=True)

result.export("triangle_soap_dish_LEFT.stl")
rresult.export("triangle_soap_dish_RIGHT.stl")
both = result + rresult

(both).export("triangle_soap_dish_BOTH.stl")

Use the multi select box below to select one or more steps you want to examine
+

+


In [ ]:
show_object(result+rresult, show_result=True)